In [1]:
import json
from pathlib import Path

import pandas as pd

In [2]:
from mhqa.dvc import load_experiments

filepaths = list(Path("../../tmp/musique-sweep/").glob("*.json"))
experiments = [exp for fp in filepaths for exp in load_experiments(fp)]
print(f"{len(experiments)} experiments")
experiments[0]

2 experiments


{'commit': 'workspace',
 'id': 'workspace',
 'name': None,
 'params': {'train': {'dataset': {'path': 'bdsaglam/musique-sweep',
    'name': 'answerable',
    'split': 'train'},
   'optimizer': 'noop',
   'ensemble': 'no'},
  'evaluation': {'dataset': {'path': 'bdsaglam/musique-sweep',
    'name': 'answerable',
    'split': 'validation'}},
  'retrieve': {'model_type': 't5', 'top_k': 3},
  'qa': {'model': 'llama-3.3-70b-tgi',
   'temperature': 0.1,
   'technique': 'multihop-decompose'},
  'run': 1},
 'metrics': {}}

In [3]:
for exp in experiments:
    train_params = exp["params"]["train"]
    if train_params.get("ensemble") is None:
        train_params["ensemble"] = "no"

experiments[0]

{'commit': 'workspace',
 'id': 'workspace',
 'name': None,
 'params': {'train': {'dataset': {'path': 'bdsaglam/musique-sweep',
    'name': 'answerable',
    'split': 'train'},
   'optimizer': 'noop',
   'ensemble': 'no'},
  'evaluation': {'dataset': {'path': 'bdsaglam/musique-sweep',
    'name': 'answerable',
    'split': 'validation'}},
  'retrieve': {'model_type': 't5', 'top_k': 3},
  'qa': {'model': 'llama-3.3-70b-tgi',
   'temperature': 0.1,
   'technique': 'multihop-decompose'},
  'run': 1},
 'metrics': {}}

In [4]:
df = pd.json_normalize(experiments).drop(columns=["commit"])
df.drop(columns=[col for col in df.columns if "fuzzy" in col], inplace=True)
print(f"{len(df)} experiments before preprocessing")
df


2 experiments before preprocessing


,id,name,params.train.dataset.path,params.train.dataset.name,params.train.dataset.split,params.train.optimizer,params.train.ensemble,params.evaluation.dataset.path,params.evaluation.dataset.name,params.evaluation.dataset.split,...,params.qa.technique,params.run,metrics.exact_match,metrics.f1,metrics.2hops.exact_match,metrics.2hops.f1,metrics.3hops.exact_match,metrics.3hops.f1,metrics.4hops.exact_match,metrics.4hops.f1
0,workspace,None,bdsaglam/musique-sweep,answerable,train,noop,no,bdsaglam/musique-sweep,answerable,validation,...,multihop-decompose,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,6fedfa259d0fe1e4c284ff0889629c6015fb4dd1,filmy-mobs,bdsaglam/musique-sweep,answerable,train,noop,no,bdsaglam/musique-sweep,answerable,validation,...,agent-decompose,1,0.486667,0.580873,0.59,0.665,0.45,0.583429,0.42,0.49419


In [5]:
mask = df["params.evaluation.dataset.path"] == "bdsaglam/musique-sweep"

df = df.loc[mask].copy()

param_cols = [col for col in df.columns if col.startswith("params.")]
metric_cols = [col for col in df.columns if col.startswith("metrics.")]
df.dropna(subset=param_cols + metric_cols, inplace=True, how="any")
df.drop_duplicates(subset=param_cols, inplace=True)

print(f"{len(df)} experiments after preprocessing")
df

1 experiments after preprocessing


,id,name,params.train.dataset.path,params.train.dataset.name,params.train.dataset.split,params.train.optimizer,params.train.ensemble,params.evaluation.dataset.path,params.evaluation.dataset.name,params.evaluation.dataset.split,...,params.qa.technique,params.run,metrics.exact_match,metrics.f1,metrics.2hops.exact_match,metrics.2hops.f1,metrics.3hops.exact_match,metrics.3hops.f1,metrics.4hops.exact_match,metrics.4hops.f1
1,6fedfa259d0fe1e4c284ff0889629c6015fb4dd1,filmy-mobs,bdsaglam/musique-sweep,answerable,train,noop,no,bdsaglam/musique-sweep,answerable,validation,...,agent-decompose,1,0.486667,0.580873,0.59,0.665,0.45,0.583429,0.42,0.49419


In [6]:
def sorted_tuple(x):
    return tuple(sorted(x))

run_counts = (
    df.groupby(param_cols[:-1])[
        "params.run"
    ]
    .aggregate(sorted_tuple)
    .reset_index()
)

In [7]:
for col in run_counts.columns:
    print(f"- {col}")
    for value in run_counts[col].unique():
        print(f"\t{value}")

- params.train.dataset.path
	bdsaglam/musique-sweep
- params.train.dataset.name
	answerable
- params.train.dataset.split
	train
- params.train.optimizer
	noop
- params.train.ensemble
	no
- params.evaluation.dataset.path
	bdsaglam/musique-sweep
- params.evaluation.dataset.name
	answerable
- params.evaluation.dataset.split
	validation
- params.retrieve.model_type
	golden
- params.retrieve.top_k
	3
- params.qa.model
	llama-3.3-70b-tgi
- params.qa.temperature
	0.1
- params.qa.technique
	agent-decompose
- params.run
	(1,)


In [8]:
df.to_json('exps.jsonl', orient='records', lines=True)

In [9]:
df[["params.qa.technique", "params.train.optimizer", "metrics.exact_match", "metrics.f1"]]

,params.qa.technique,params.train.optimizer,metrics.exact_match,metrics.f1
1,agent-decompose,noop,0.486667,0.580873
